In [1]:
# !pip install dicttoxml

In [2]:
import psutil # мониторинг системных ресурсов
import os # взаимодействование с операционной системой
import json # работа с данными JSON
import dicttoxml # преобразование словарей в XML
import zipfile # работа со сжатыми файлами и папками

In [3]:
# константы
FILENAME = "example.txt"
FILENAME_JSON = "example.json"
FILENAME_XML = "example.xml"
ARCHIVE_NAME = 'test'

In [4]:
# создание файла
def create_file(filename):
    if os.path.exists(filename):
        print(f"Файл {filename} уже сущетвует")
        return

    with open(filename, "w") as f:
        print(f"Файл {filename} Создан")

In [5]:
# удаление файла
def delete_file(filename):
    if not os.path.exists(filename):
        print(f"Файла {filename} не существует")
        return

    try:
        os.remove(filename)
        print(f"Файл {filename} удален")
    except OSError as e:
        print(f"Ошибка удаления файла {filename}: {e}")

In [6]:
# запись в файл
def write_to_file(filename):
    text = input(f"Введите текст, который будет записан в файл {filename}: ")

    try:
        with open(filename, 'w') as file:
            file.write(text)
            print('Текст успешно записан в файл')
    except Exception as e:
        print(f"Произошла ошибка записи текста в файл: {e}")

In [8]:
# чтение файла
def read_the_file(filename):
    try:
        with open(filename, "r") as file:
            text = file.read()
            print(f"Текст в файле {filename}: {text}")
    except Exception as e:
        print(f"Произошла ошибка при чтении файла: {e}")

----
#`Задание 1`

Вывести информацию в консоль о логических дисках, именах, метке тома, размере типе файловой системы.

In [14]:
def display_logical_disks_info():
    drives = psutil.disk_partitions()

    if not drives:
        print("Дисков не обнаружено")
    else:
        for drive in drives:
            print(f"Диск: {drive.device}")
            print(f"Метка тома: {drive.mountpoint}")
            print(f"Тип файловой системы: {drive.fstype}")
            try:
                drive_usage = psutil.disk_usage(drive.mountpoint)
            except PermissionError:
                continue
            print(f"Размер диска: {drive_usage.total}")
            print(f"Свободное пространство: {drive_usage.free}")
            print()

----
#`Задание 2`

Работа с файлами
- Создать файл
- Записать в файл строку
- Прочитать файл в консоль
- Удалить файл

In [10]:
def file():
    create_file(FILENAME)
    write_to_file(FILENAME)
    read_the_file(FILENAME)

----
#`Задание 3`
Работа с форматом JSON
- Создать файл формате JSON
- Создать новый объект
- Выполнить сериализацию объекта в формате JSON и записать в файл
- Прочитать файл в консоль
- Удалить файл

#`Задание 4`
Работа с форматом XML
- Создать файл формате XML из редактора
- Записать в файл новые данные из консоли
- Прочитать файл в консоль
- Удалить файл

In [20]:
# для создания объектов-людей
class Person:
    def __init__(self, name : str, age : int):
        # Инициализируем человека с заданным именем и возрастом
        self.name = name
        if int(age) < 0:
            self.age = 0
        else:
            self.age = int(age)

    # Возвращаем словарное представление объекта person
    def asDict(self):
        return {'Name': self.name,'Age': self.age}

# базовый класс для сохранения людей
class Driver:
    def Save(self, persons:list, filename):
        pass

# для сохранения людей в формате XML
class XMLDriver(Driver):
    def Save(self, persons: list, filename):
        # Инициализируем пустой словарь для хранения данных XML
        xmlData = { 'Persons' : [] };

        # Перебираем список людей и конвертируем каждого человека в словарь
        for person in persons:
            xmlData['Persons'].append(person.asDict())

        # Записываем данные XML в файл
        with open(filename,"w") as my_file:
            my_file.write(dicttoxml.dicttoxml(xmlData,return_bytes=False))

# для сохранения людей в формате JSON
class JSONDriver(Driver):
    def Save(self, persons:list, filename):
        # Инициализируем пустой список для хранения данных JSON
        jsonData = [];

        # Перебираем список людей и конвертируем каждого человека в словарь
        for person in persons:
            jsonData.append(person.asDict())

        # Записываем данные JSON в файл
        with open(filename,"w") as my_file:
            json.dump(jsonData, my_file)

# для управления данными о людях
class PersonHandler():
    def __init__(self):
        # Инициализируем пустой список для хранения людей
        self.persons = []

    def SavePersons(self, d: Driver, filename):
        # Вызоваем метод Save объекта Driver, чтобы сохранить людей в файл
        d.Save(self.persons, filename)

    def AddPerson(self, name : str, age : int):
        # Создаем новый объект Person с указанным именем и возрастом и добавляем его в список людей
        self.persons.append(Person(name,age))

    def FindPerson(self):
        # Выведим имя и возраст каждого человека в списке людей
        [print(f"Name: {item.name}, Age: {item.age}") for item in self.persons]

    def DeletePerson(self, name: str):
        # Перебираем список людей и удаляем человека с указанным именем
        for person in self.persons:
            if name in person.name:
                self.persons.remove(person)
                print(f'персонаж {name} успешно удалена')
            else:
                print(f'персоны {name} не существует ')

class Program():
    @staticmethod
    def menu():
        # Отображение меню с опциями взаимодействия с данными человека
        print('1. Добавить person\n'
              '2. удалить person\n'
              '3. посмотреть персонажей\n'
              '4. JSON\n'
              '5. XML\n'
              '0. Выход')

    @staticmethod
    def SaveXML(handler):
        # Сохраняем данные о человеке в XML-файл, используя PersonHandler
        driver = XMLDriver()
        handler.SavePersons(driver, FILENAME_XML)

    @staticmethod
    def SaveJSON(handler):
        # Сохраняем данные о человеке в файл JSON, используя PersonHandler
        driver = JSONDriver()
        handler.SavePersons(driver, FILENAME_JSON)

    @staticmethod
    def AddPerson(handler : PersonHandler):
        # Добавляем нового человека, используя данные пользователя
        name = input("Имя: ")
        age = input("Возраст: ")
        handler.AddPerson(name, age)

    @staticmethod
    def DeletePerson(handler : PersonHandler):
        # Удаление человека используя данные пользователя
        name = input("Имя: ")
        handler.DeletePerson(name)

    @staticmethod
    def FindPerson(handler : PersonHandler):
        # Отображение данных обо всех людях
        handler.FindPerson()

    @staticmethod
    def main():
        # Основной метод запуска программы
        handler = PersonHandler()

        while True:
            Program.menu()
            for_case = input("Выбери действие: ")

            # добавить персонажа
            if for_case == "1":
                Program.AddPerson(handler)

            # удаление персонажа
            elif for_case == "2":
                check = input("удалить person? (1 - да / 0 - нет): ")
                if check == "1":
                    Program.DeletePerson(handler)
                elif check == "0":
                    pass
                else:
                    print('err')

            # посмотреть персонаже
            elif for_case == "3":
                Program.FindPerson(handler)

            # работа json
            elif for_case == "4":
                check = input("Создать/изменить - 2 \ удалить JSON файл - 1 \ отмена - 0: ")
                if check == "2":
                    Program.SaveJSON(handler)
                elif check == '1':
                    delete_file(FILENAME_JSON)
                elif check == "0":
                    pass
                else:
                    print('выбор не из меню')

            # работа с xml
            elif for_case == "5":
                check = input("Создать/изменить - 2 \ удалить XML файл - 1 \ отмена - 0: ")
                if check == "2":
                    Program.SaveXML(handler)
                elif check == '1':
                    delete_file(FILENAME_XML)
                elif check == '0':
                    pass
                else:
                    print('выбор не из меню')

            # выход
            elif for_case == "0":
                break

----
#`Задание 5`
Создание zip архива, добавление туда файла, определение размера архива
- Создать архив в форматер zip
- Добавить файл в архив
- Разархивировать файл и вывести данные о нем
- Удалить файл и архив

In [21]:
def zip_file():
    def create_zip_archive(filename, archive_name):
        # Создает ZIP-архив из файла
        with zipfile.ZipFile(archive_name, 'w') as zip_file:
            zip_file.write(filename)

    def get_archive_size(arhivename):
        # Получает размер ZIP-архива
        size = os.path.getsize(arhivename)
        print(f"Размер архива: {size} байт")

    def extract_zip_archive(archive_name):
        # Извлекает содержимое ZIP-архива
        with zipfile.ZipFile(archive_name, 'r') as zip_file:
            zip_file.extractall()
        file_name = os.path.splitext(archive_name)[0]

        print(f"Извлечен файл: {file_name}")

    create_file(FILENAME)
    write_to_file(FILENAME)
    create_zip_archive(FILENAME, ARCHIVE_NAME)
    get_archive_size(ARCHIVE_NAME)
    extract_zip_archive(ARCHIVE_NAME)

----

In [22]:
from IPython.display import clear_output

for_case = ""

while(for_case != '0'):

    for_case = input('\n\n1. Посмотреть информацию о дисках\n'
                     '2. Работа с файлами\n'
                     '3. Работа с JSON или XML\n'
                     '4. Работа с ZIP\n'
                     '0. Выход\n')

    clear_output()

    # Задание 1
    if for_case == '1':
        display_logical_disks_info()

    # Задание 2
    elif for_case == '2':
        check_create = input('Создать файл? (1 - да / 0 - нет): ')
        if check_create == '1':
            file()

            check_del = input("Удалить файл? (1 - да / 0 - нет): ")
            if check_del == "1":
                delete_file(FILENAME)
            elif check_del == '0':
                print("Нет, так нет(")
            else:
                print('err')

        elif check_create == '0':
            print('Нет, так нет(')
        else:
            print('err')

    # Задание 3\4
    elif for_case == "3":
        Program.main()

    # Задание 5
    elif for_case == "5":
        check_create =  input("Создать zip архив? (1 - да / 0 - нет): ")
        if check_create == "1":
            zip_file()

            check_del = input("Удалить zip архив? (1 - да / 0 - нет): ")
            if check_del == "1":
                delete_file(FILENAME)
                delete_file(ARCHIVE_NAME)
            elif check_del == '0':
                print("Нет, так нет(")
            else:
                print('err')

        elif check_create == '0':
            print("Нет, так нет(")
        else:
            print('err')

    elif for_case =='0':
        print("Выход")
        break

    else:
        print("Выбор не из меню.")
        break

Выход
